In [8]:
import deal
import math
from typing import List

distance_function = deal.chain(
    deal.pre(
        lambda a, b:
            a is not None and b is not None
            and isinstance(a, (list, tuple)) and isinstance(b, (list, tuple))
            and 0 < len(a) == len(b)
            and all(isinstance(x, (int, float)) for x in a)
            and all(isinstance(y, (int, float)) for y in b)
    ),
    deal.post(lambda r: isinstance(r, (int, float))),
    deal.post(lambda r: r >= 0),
    deal.pure
)

@distance_function
def euclidean_distance(a: List[float], b: List[float]) -> float:
    return math.sqrt(sum((yi - xi) ** 2 for xi, yi in zip(a, b)))

@distance_function
def manhattan_distance(a: List[float], b: List[float]) -> float:
    return sum(abs(yi - xi) for xi, yi in zip(a, b))


In [9]:
from hypothesis import given, strategies as st
import pytest

@st.composite
def list_pairs(draw):
    length = draw(st.integers(min_value=1, max_value=10))
    floats = st.floats(allow_nan=False, allow_infinity=False)
    a = draw(st.lists(floats, min_size=length, max_size=length))
    b = draw(st.lists(floats, min_size=length, max_size=length))
    return a, b

@given(list_pairs())
def test_commutativity_euclidean(pair):
    print(pair)
    a, b = pair
    assert euclidean_distance(a, b) == pytest.approx(euclidean_distance(b, a))

@given(list_pairs())
def test_commutativity_manhattan(pair):
    a, b = pair
    assert manhattan_distance(a, b) == manhattan_distance(b, a)

Trying:
    with raises(ZeroDivisionError):
        1 / 0
Expecting nothing
**********************************************************************
File "__main__", line 6, in __main__.raises
Failed example:
    with raises(ZeroDivisionError):
        1 / 0
Expected nothing
Got:
    Excepted: division by zero
6 items had no tests:
    __main__
    __main__.euclidean_distance
    __main__.list_pairs
    __main__.manhattan_distance
    __main__.test_commutativity_euclidean
    __main__.test_commutativity_manhattan
**********************************************************************
1 item had failures:
   1 of   1 in __main__.raises
1 test in 7 items.
0 passed and 1 failed.
***Test Failed*** 1 failure.


TestResults(failed=1, attempted=1)